In [3]:
using Distributed, ApproxFun, ApproxFunRational, Printf, ApproxFunOrthogonalPolynomials, ApproxFunFourier, AbstractIterativeSolvers
addprocs(2) # Note: This was originally run on a cluster with 40 cores.

LoadError: ArgumentError: Package ApproxFun [28f2ccd6-bb30-5033-b560-165f7b14dc2f] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


In [2]:
@everywhere using ApproxFun, ApproxFunRational, ApproxFunOrthogonalPolynomials, ApproxFunFourier, AbstractIterativeSolvers
@everywhere prec = 170
@everywhere sp(x) = BigFloat(x,prec)
@everywhere spc(x) = BigFloat(real(x),prec)+1im*BigFloat(imag(x),prec)

In [3]:
@everywhere function to_Mathematica(x)
    data = split(x,"e")
    if length(data) < 2
        return x
    else
        return data[1] * "*10^(" * data[2] * ")"
    end
end

@everywhere function to_BigFloat(x::String,prec::Int64)
    datasplit = split(x,"`")
    if length(datasplit) < 2
        if x == "Indeterminate"
            return BigFloat("0",prec)
        else
            return BigFloat(x,prec)
        end
    end
    datasplit2 = split(datasplit[2],"^")
    str = datasplit[1]
    if length(datasplit2) > 1
        str *= "e" * datasplit2[end]
    end
    return BigFloat(str,prec)
end

In [4]:
@everywhere dom = PeriodicLine{0,BigFloat}(0)
@everywhere α = sp("0.0");
@everywhere Ftest = Fun(cai(x -> exp(-x^2),sp("0")),OscLaurent{PeriodicLine{0,BigFloat},BigFloat}(dom,α),4096)

@everywhere pts = map(string,real(points(Ftest)))
@everywhere pts = map(to_Mathematica,pts)
@everywhere open("points.txt", "w") do f
  for i in pts
    println(f, i)
  end
end

## Now run Mathematica code (ComputeR.nb) to generate relfection coefficient data

Note: This is for $q(x,0) = U_0 \mathrm{sech}^2(x)$ where $U_0 < 0$.  To change $U_0$, `ComputeR.nb` needs to be edited.

In [5]:
@everywhere real_r = map(z -> to_BigFloat(z,prec), readlines("reflection_re.txt") )
@everywhere imag_r = map(z -> to_BigFloat(z,prec), readlines("reflection_im.txt") )
@everywhere r = real_r + imag_r*1im;

In [6]:
@everywhere function KdV(x,t,tol,flag = false)
    α = sp("2")*sp(x);
    Fsp = OscLaurent{PeriodicLine{0,BigFloat},BigFloat}(dom,α)
    pts = real(points(Ftest))
    if t != 0
        rt = exp.(sp("8")*1im*pts.^3*sp(t)).*r
    else
        rt = r
    end
    rtx = rt.*(2im*pts)
    
    F = Fun(Fsp,ApproxFun.transform(Fsp,rt))
    Fx = Fun(Fsp,ApproxFun.transform(Fsp,rtx))
    
    if flag
        print("error estimate = ")
        println(BigFloat( abs.(coefficients(Fx)[end-10:end]) |> sum , 3))
    end
    
    cF = conj(copy(F))
    cFx = conj(copy(Fx))
    h = map(SumFun,[F,-cF])
    hx = map(SumFun,[Fx,-cFx])
    
    simp(f) = chop(combine!(chop(f,tol)),tol);
    h = simp(h)
    
    𝓒⁺ = Cauchy(+1)
    𝓒⁻ = Cauchy(-1)
    function S(x)
        out = copy(x)
        out[1] -= (𝓒⁺*x[2])*F
        out[2] += (𝓒⁻*x[1])*cF
        return out
    end
    
    function Sx(x)
        out = copy(x)
        out[1] = -(𝓒⁺*x[2])*Fx
        out[2] = (𝓒⁻*x[1])*cFx
        return out
    end
    
    out = GMRES_hi(S,h,⋅,tol,30,x -> simp(x),prec,flag)
    sol = +([out[2][i]*out[1][i] for i=1:length(out[2])]...)
    
    q = (sum(sol)/big(pi))[2]
    
    b = simp(hx - Sx(sol))
    out = GMRES_hi(S,b,⋅,tol,30,x -> simp(x),prec,flag)
    solx = +([out[2][i]*out[1][i] for i=1:length(out[2])]...)
    
    #println("Done.")
    qx = (sum(solx)/big(pi))[2]
    
    [q,qx]
end

@everywhere function KdVsym(x,t,tol)
    print(x[2])
    print("  ")
    println(BigFloat(x[1],6))
    if x[1] >= 0
        return KdV(x[1],t,tol)
    else
        return KdV(-x[1],-t,tol)
    end
end

In [7]:
N = 2048
grid = (2*map(sp,0:N-1 |> Array)/big(N) .-1)*big(pi)/sp("2")
grid = tan.(grid);

In [8]:
pts = map(string,real(grid))
open("2048_grid.txt", "w") do f
  for i in pts
    println(f, i)
  end
end

In [9]:
grid_num = [ (grid[i],i) for i in 1:N ];

In [11]:
@everywhere f = j -> KdVsym(j,sp("1e-4"),1e-50)
data = pmap(f,grid_num);

      From worker 32:	31        From worker 33:	34  -19.5
      From worker 35:	33  -20.5
      From worker 34:	32  -21.0
      From worker 32:	-21.5
      From worker 18:	17  -41.0
      From worker 10:	9  -82.0
      From worker 28:	27  -25.0
      From worker 2:	1  -1.81e+77
      From worker 26:	25  -27.0
      From worker 36:	36  -18.5
      From worker 37:	35  -19.0
      From worker 27:	26  -26.0
      From worker 5:	3  -328.0
      From worker 4:	8  -94.0
      From worker 6:	5  -164.0
      From worker 9:	7  -108.0
      From worker 7:	4  -216.0
      From worker 3:	2  -656.0
      From worker 8:	6  -132.0
      From worker 2:	37  -18.0
      From worker 33:	38  -17.5
      From worker 32:	46  -14.5
      From worker 14:	10  -72.0
      From worker 12:	13  -54.0
      From worker 12:	50  -13.2
      From worker 15:	15  -47.0
      From worker 13:	12  -59.0
      From worker 11:	11  -66.0
      From worker 17:	14  -50.0
      From worker 16:	16  -43.0
      From worker 10:	51  

      From worker 16:	232  -2.69
      From worker 16:	267  -2.31
      From worker 11:	236  -2.62
      From worker 17:	237  -2.62
      From worker 12:	257  -2.44
      From worker 13:	261  -2.38
      From worker 15:	262  -2.38
      From worker 14:	264  -2.31
      From worker 10:	269  -2.31
      From worker 29:	241  -2.56
      From worker 30:	242  -2.56
      From worker 31:	243  -2.56
      From worker 28:	271  -2.25
      From worker 7:	249  -2.5
      From worker 7:	285  -2.12
      From worker 6:	265  -2.31
      From worker 8:	276  -2.25
      From worker 5:	277  -2.25
      From worker 4:	280  -2.19
      From worker 9:	282  -2.19
      From worker 3:	283  -2.19
      From worker 2:	286  -2.12
      From worker 26:	292  -2.06
      From worker 37:	293  -2.06
      From worker 27:	294  -2.06
      From worker 22:	268  -2.31
      From worker 20:	284  -2.19
      From worker 19:	287  -2.12
      From worker 21:	288  -2.12
      From worker 24:	289  -2.12
      From worker 25

      From worker 36:	517  -0.984
      From worker 35:	484  -1.09
      From worker 33:	485  -1.09
      From worker 34:	492  -1.06
      From worker 32:	518  -0.984
      From worker 29:	489  -1.06
      From worker 30:	515  -1.0
      From worker 31:	522  -0.969
      From worker 28:	523  -0.969
      From worker 22:	508  -1.0
      From worker 21:	529  -0.953
      From worker 19:	532  -0.938
      From worker 25:	533  -0.938
      From worker 20:	534  -0.938
      From worker 23:	535  -0.938
      From worker 24:	537  -0.922
      From worker 18:	540  -0.922
      From worker 8:	524  -0.969
      From worker 5:	526  -0.953
      From worker 6:	527  -0.953
      From worker 9:	536  -0.938
      From worker 3:	539  -0.922
      From worker 7:	541  -0.922
      From worker 4:	543  -0.906
      From worker 2:	547  -0.906
      From worker 26:	548  -0.891
      From worker 27:	549  -0.891
      From worker 16:	519  -0.984
      From worker 13:	530  -0.953
      From worker 15:	531  -0.

      From worker 9:	750  -0.445
      From worker 6:	751  -0.445
      From worker 8:	752  -0.445
      From worker 5:	753  -0.445
      From worker 3:	755  -0.438
      From worker 7:	756  -0.438
      From worker 4:	758  -0.438
      From worker 2:	765  -0.422
      From worker 29:	740  -0.469
      From worker 30:	761  -0.43
      From worker 31:	771  -0.414
      From worker 28:	773  -0.406
      From worker 37:	776  -0.398
      From worker 22:	747  -0.453
      From worker 21:	768  -0.414
      From worker 24:	769  -0.414
      From worker 25:	770  -0.414
      From worker 20:	775  -0.406
      From worker 23:	777  -0.398
      From worker 19:	779  -0.398
      From worker 18:	780  -0.398
      From worker 36:	781  -0.391
      From worker 12:	772  -0.406
      From worker 15:	778  -0.398
      From worker 13:	782  -0.391
      From worker 14:	784  -0.391
      From worker 11:	788  -0.383
      From worker 17:	792  -0.375
      From worker 16:	795  -0.367
      From worker 10:	7

      From worker 34:	995  -0.0459
      From worker 33:	1008  -0.0259
      From worker 35:	1015  -0.0154
      From worker 32:	1016  -0.0139
      From worker 30:	985  -0.0615
      From worker 29:	986  -0.0596
      From worker 31:	1017  -0.0122
      From worker 28:	1019  -0.00928
      From worker 9:	1000  -0.0381
      From worker 6:	1001  -0.0371
      From worker 3:	1003  -0.0342
      From worker 8:	1005  -0.0308
      From worker 7:	1007  -0.0278
      From worker 4:	1011  -0.0215
      From worker 5:	1012  -0.020
      From worker 2:	1022  -0.00464
      From worker 22:	994  -0.0479
      From worker 21:	1006  -0.0293
      From worker 24:	1009  -0.0244
      From worker 25:	1013  -0.0186
      From worker 20:	1014  -0.0171
      From worker 19:	1018  -0.0107
      From worker 18:	1025  0.0
      From worker 27:	1029  0.0061
      From worker 26:	1031  0.00928
      From worker 23:	1026  0.00153
      From worker 22:	1028  0.00464
      From worker 18:	1035  0.0154
      Fro

      From worker 12:	1217  0.305
      From worker 14:	1225  0.32
      From worker 15:	1226  0.32
      From worker 13:	1230  0.328
      From worker 11:	1239  0.344
      From worker 17:	1242  0.344
      From worker 16:	1247  0.352
      From worker 10:	1249  0.359
      From worker 37:	1263  0.383
      From worker 36:	1266  0.391
      From worker 31:	1234  0.336
      From worker 30:	1237  0.336
      From worker 29:	1267  0.391
      From worker 28:	1268  0.391
      From worker 34:	1241  0.344
      From worker 33:	1256  0.367
      From worker 35:	1271  0.398
      From worker 32:	1274  0.398
      From worker 9:	1251  0.359
      From worker 6:	1255  0.367
      From worker 3:	1257  0.375
      From worker 8:	1261  0.375
      From worker 7:	1269  0.391
      From worker 4:	1273  0.398
      From worker 5:	1276  0.406
      From worker 2:	1279  0.414
      From worker 21:	1248  0.359
      From worker 24:	1250  0.359
      From worker 25:	1253  0.367
      From worker 20:	12

      From worker 9:	1471  0.812
      From worker 6:	1475  0.828
      From worker 3:	1479  0.844
      From worker 8:	1483  0.844
      From worker 7:	1486  0.859
      From worker 4:	1489  0.859
      From worker 5:	1493  0.875
      From worker 2:	1499  0.891
      From worker 27:	1504  0.906
      From worker 26:	1510  0.922
      From worker 30:	1480  0.844
      From worker 31:	1484  0.844
      From worker 28:	1511  0.922
      From worker 34:	1485  0.859
      From worker 33:	1508  0.922
      From worker 35:	1513  0.922
      From worker 32:	1518  0.938
      From worker 21:	1490  0.859
      From worker 24:	1494  0.875
      From worker 20:	1496  0.875
      From worker 25:	1498  0.891
      From worker 19:	1500  0.891
      From worker 22:	1503  0.906
      From worker 23:	1506  0.906
      From worker 18:	1524  0.953
      From worker 12:	1497  0.891
      From worker 14:	1501  0.891
      From worker 15:	1502  0.891
      From worker 13:	1505  0.906
      From worker 11:	

      From worker 24:	1732  1.91
      From worker 20:	1740  1.94
      From worker 25:	1742  1.97
      From worker 19:	1745  2.0
      From worker 22:	1749  2.0
      From worker 23:	1751  2.06
      From worker 21:	1757  2.06
      From worker 18:	1758  2.06
      From worker 35:	1730  1.88
      From worker 33:	1755  2.06
      From worker 32:	1759  2.12
      From worker 30:	1725  1.84
      From worker 31:	1726  1.84
      From worker 29:	1728  1.88
      From worker 28:	1762  2.12
      From worker 6:	1731  1.88
      From worker 9:	1734  1.91
      From worker 3:	1735  1.91
      From worker 4:	1736  1.91
      From worker 7:	1737  1.94
      From worker 8:	1738  1.94
      From worker 5:	1744  1.97
      From worker 2:	1763  2.12
      From worker 27:	1775  2.25
      From worker 15:	1746  2.0
      From worker 14:	1747  2.0
      From worker 13:	1748  2.0
      From worker 11:	1750  2.0
      From worker 17:	1754  2.06
      From worker 16:	1756  2.06
      From worker 12:	17

      From worker 37:	1998  12.8
      From worker 20:	1973  8.5
      From worker 19:	1988  10.8
      From worker 25:	1989  10.8
      From worker 22:	1992  11.5
      From worker 21:	1993  11.5
      From worker 24:	1994  11.8
      From worker 18:	2000  13.2
      From worker 27:	2002  13.8
      From worker 12:	1983  9.75
      From worker 14:	1987  10.5
      From worker 11:	1990  11.0
      From worker 16:	1996  12.2
      From worker 15:	1999  13.0
      From worker 10:	2006  15.2
      From worker 29:	1997  12.5
      From worker 31:	2001  13.5
      From worker 28:	2007  15.5
      From worker 35:	1980  9.5
      From worker 35:	2008  16.0
      From worker 34:	2005  14.8
      From worker 32:	2017  20.5
      From worker 26:	2020  22.5
      From worker 36:	2027  29.5
      From worker 9:	2009  16.5
      From worker 7:	2010  16.5
      From worker 3:	2012  17.5
      From worker 5:	2014  18.5
      From worker 6:	2018  21.0
      From worker 4:	2022  24.0
      From worker 

In [2]:
using Plots, JLD2, FileIO, LaTeXStrings

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


ArgumentError: ArgumentError: Package JLD2 not found in current path:
- Run `import Pkg; Pkg.add("JLD2")` to install the JLD2 package.


In [ ]:
FileIO.save("hiprectest.jld2","data",data)

In [ ]:
function pairup(x,j)
    y = (0*x[1:2:end] .+ j,x[2:2:end])
end

In [ ]:
b = FileIO.load("hiprectest.jld2","data");

In [ ]:
p = plot(pairup(b[1][end],1), yaxis = :log, seriestype = :scatter, 
    ylabel = latexstring("\\mathrm{GMRES~Residual}"), markercolor=:black, xlabel = latexstring("x"), framestyle=:box,minorgrid=:true,tickfont = font(10),legendfontsize = 10,guidefontsize=14)
for j = 2:40
    plot!(pairup(b[j][end],j), yaxis = :log, seriestype = :scatter, markercolor=:black, legend = false, framestyle = :box)
end
display(p)

In [ ]:
n = 120 #length(b)
q = [b[i][2] for i = 1:n];
prec = 170
sp(x) = BigFloat(x,prec)
spc(x) = BigFloat(real(x),prec)+1im*BigFloat(imag(x),prec)
qreal = [ -sech( sp(i*1.0) )^2 for i = 0:n-1]
err = q-qreal
p = plot(0:n-1, abs.(q-qreal |> real), yaxis = ([1e-56,1e-47],:log), lw =2, legend = false, ylabel = latexstring("\\mathrm{Reconstruction~Error}"),xlabel = latexstring("x"), framestyle=:box,minorgrid=:true,tickfont = font(10),legendfontsize = 10,guidefontsize=14)
p |> display